# web_data_flow walkthrough（细讲版）

## 目标与先修
- 目标：观察前端请求数据如何在 FastAPI 中校验、计算并返回 JSON。
- 先修：了解 HTTP POST、JSON body、状态码。


## 流程总览
1. 准备请求 payload。
2. 本地预计算订单汇总。
3. 通过 `TestClient` 调用 API。
4. 观察正常分支与错误分支，再对照 `run_demo`。


### 环境检查（离线模块）
- 本步做什么：确认本模块不依赖外部 API key，直接验证本地依赖可导入。
- 为什么这样做：保证 notebook 能在离线环境稳定 Run All。
- 输入：本地 Python 运行环境与项目依赖。
- 输出：导入成功提示。
- 观察点：如果失败，优先检查 `requirements.txt` 和当前虚拟环境。


In [ ]:
from fastapi.testclient import TestClient
from component import calc_order_summary, create_app, run_demo

print("环境检查通过：离线依赖已成功导入")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 构造请求 payload
- 本步做什么：定义订单条目和税率。
- 为什么这样做：后续本地计算与 API 计算都复用同一输入，便于对照。
- 输入：商品行项目和税率。
- 输出：`request_payload`。
- 观察点：注意 `qty` 与 `unit_price` 是关键计算字段。


In [ ]:
request_payload = {
    "items": [
        {"sku": "Keyboard", "qty": 2, "unit_price": 49.5},
        {"sku": "Mouse", "qty": 1, "unit_price": 25.0},
    ],
    "tax_rate": 0.1,
}
show_json(request_payload)


### Step 2: 本地预计算
- 本步做什么：调用 `calc_order_summary(...)` 做本地计算。
- 为什么这样做：建立“期望结果”，用于与 API 返回对照。
- 输入：`items` 与 `tax_rate`。
- 输出：subtotal/tax/total。
- 观察点：本地结果应与 API 成功返回一致。


In [ ]:
local_summary = calc_order_summary(request_payload["items"], request_payload["tax_rate"])
show_json(local_summary)


### Step 3: 调用 API 成功分支
- 本步做什么：创建 app 和 `TestClient`，调用 `/api/orders/summary`。
- 为什么这样做：验证真实请求链路（序列化/校验/处理/响应）。
- 输入：`request_payload`。
- 输出：HTTP 响应对象。
- 观察点：看 `status_code`、响应头、`response.json()`。


In [ ]:
app = create_app()
with TestClient(app) as client:
    response = client.post("/api/orders/summary", json=request_payload)

show_kv("成功分支响应", {"status_code": response.status_code, "content_type": response.headers.get("content-type")})
show_json(response.json())


### Step 4: 调用 API 错误分支
- 本步做什么：构造非法 payload（例如 `qty=0`）并再次请求。
- 为什么这样做：教学必须覆盖失败路径，说明数据校验如何触发。
- 输入：`bad_payload`。
- 输出：错误响应（4xx）。
- 观察点：关注错误结构与字段提示，理解请求为何被拒绝。


In [ ]:
bad_payload = {"items": [{"sku": "Broken", "qty": 0, "unit_price": 10}], "tax_rate": 0.1}

with TestClient(app) as client:
    bad_response = client.post("/api/orders/summary", json=bad_payload)

show_kv("错误分支响应", {"status_code": bad_response.status_code})
show_json(bad_response.json())


## 端到端结果

### Step 5: 运行 run_demo
- 本步做什么：执行 `run_demo()` 获取统一 trace。
- 为什么这样做：确认手动流程与模块演示流程一致。
- 输入：无。
- 输出：`demo_result`。
- 观察点：trace 中应能看到请求数据、计算结果和响应结果。


In [ ]:
demo_result = run_demo()
show_json(demo_result)


## 常见错误
- `422` 校验失败：payload 字段类型或范围不合法。
- 本地计算和 API 结果不一致：优先检查输入是否完全相同。
- 在 notebook 里直接起服务导致阻塞：教学场景优先使用 `TestClient`。

## 总结
- Web 数据流核心是“请求结构 + 校验 + 业务计算 + 标准响应”。
- 通过成功/失败双分支观察，能更快定位接口问题。
